# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;

- Строят модель и оценивают объём запасов;

- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;

- Определяют регион с максимальной суммарной прибылью отобранных скважин (прибыль равна суммарной прибыли отобранных месторождений).

# Описание данных

Данные геологоразведки трёх регионов находятся в соответствующих файлах. 

- `id` — уникальный идентификатор скважины;

- `f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы);

- `product` — объём запасов в скважине (тыс. баррелей).

# Условия задачи

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).

- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

- Бюджет на разработку скважин в регионе — 10 млрд рублей.

- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

In [1]:
#!pip install sweetviz

In [2]:
# Подключение библиотек

import warnings

import numpy as np
import pandas as pd
import sweetviz as sv
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression # линейная регрессия
from sklearn.dummy import DummyRegressor # для проверки модели на адекватность

from sklearn.model_selection import train_test_split # для разделения выборки
from sklearn.preprocessing import StandardScaler # для масштабирования признаков
from sklearn.utils import shuffle

from sklearn.metrics import mean_squared_error # метрики

# Настройки
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
# Константы

STATE = 123 # значение для параметра random_state

BUDGET = 10 * (10 ** 9) # бюджет на разработку скважин в регионе, руб.
UNIT_INCOME = 450000 # доход с каждой единицы продукта, руб.
NUM_WELLS = 200 # требуемое кол-во точек для разработки, шт.

N_SAMP = 1000 # кол-во выборок для техники Bootstrap
N = 500 # кол-во точек при разведке региона

In [4]:
# Функция для формирования информационных отчетов по каждому региону

def inform_report(list_df):
    list_reports = []
    list_names = []
    
    for num in range(len(list_df)):
        
        title = 'Geo_data_' + str(num)
        report = sv.analyze([list_df[num], title])
        report_name = 'Common_analysis_' + str(num) + '.html'
        
        list_reports.append(report)
        list_names.append(report_name)
    
    return dict(zip(list_names, list_reports))

In [5]:
# Функция для стандартизации признаков

def scaler_features(features_train, features_valid):
    
    scaler = StandardScaler()
    scaler.fit(features_train)
    
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)
    
    return features_train, features_valid

In [6]:
# Функция для построения модели (для каждого региона)

def lr_model(features_train, features_valid, target_train, target_valid):
    
    lr = LinearRegression()
    lr.fit(features_train, target_train)
    
    target_pred = lr.predict(features_valid)
    score = mean_squared_error(target_valid, target_pred) ** 0.5
    
    return lr_model, target_pred, score   

In [7]:
# Функция для построения модели DummyRegressor (для проверки моделей на адекватность)

def dr_model(features_train, features_valid, target_train, target_valid, value, strategy='constant'):
    
    dr = DummyRegressor(strategy=strategy, constant=value)
    dr.fit(features_train, target_train)
    
    score = mean_squared_error(target_valid, dr.predict(features_valid)) ** 0.5
    
    return score   

## Загрузка и подготовка данных

In [8]:
try:
    df_0 = pd.read_csv('geo_data_0.csv')
    df_1 = pd.read_csv('geo_data_1.csv')
    df_2 = pd.read_csv('geo_data_2.csv')
except:
    df_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
    df_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
    df_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')
    
df_list = [df_0, df_1, df_2]

In [9]:
# Формируем отчёты по каждому региону

for key, value in inform_report(df_list).items():
    value.show_html(key)

                                             |          | [  0%]   00:00 -> (? left)

                                             |          | [  0%]   00:00 -> (? left)

                                             |          | [  0%]   00:00 -> (? left)

Report Common_analysis_0.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
Report Common_analysis_1.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
Report Common_analysis_2.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


#### Проанализировав отчёты можно сказать следующее:

Количество строк и столбцов каждого датафрейма: 100_000 и 5, соответственно. Все признаки, за исключением признака `id`, являются численными. Пропусков и полных дубликатов ни в одном датафрейме нет. Однако, в признаке `id` наблюдается небольшой процент дубликатов (менее 1%) — далее удалим соотв. строки, каждый раз оставляя последнюю запись (возможно есть привязка к дате сбора информации и последняя запись самая актуальная).

Также, по двум регионам наблюдается небольшой процент скважин с нулевым значение в признаке `product` — удалим соответствующие записи, по одному региону этот процент выше (около 8%), однако, в сравнении с размером датафрейма есть основания полагать данный набор записей незначительным, также удалим такие записи. 

Мультиколлинеарности признаков не обнаружено. Однако, стоит отметить, что для каждого региона, признак `f2` наиболее коррелирован с целевым, чем остальные, причем, для региона `geo_1` коэффициент корреляция составляет ровно 1.

In [10]:
# Удаляем дубликаты в признаке id

for i in range(len(df_list)):
    df_list[i] = df_list[i].drop_duplicates(subset='id', keep='last')

In [11]:
# Удаляем строки, соотв. нулевому значению в признаке product

for i in range(len(df_list)):
    df_list[i] = df_list[i][df_list[i]['product'] != 0]

In [12]:
# Проверка

for item in df_list:
    print(len(item[item['product'] == 0]))

0
0
0


## Обучение и проверка модели

In [13]:
# Сохраним признаки и целевой признак (далее ц.п.) в отдельных переменных
# Введём обозначения: X - список признаков по каждому региону, y - список ц.п. по каждому региону
# Признак id уберём из рассмотрения (по всем регионам)

X = []
y = []

for i in range(len(df_list)):
    X.append(df_list[i].drop(columns=['id', 'product'])) # извлекаем признаки
    y.append(df_list[i]['product']) # извлекаем целевой признак

In [14]:
# Разбиваем данные по каждому региону на обучающую и валидационную выборки в соотношении 75:25

X0_train, X0_valid, y0_train, y0_valid = train_test_split(X[0], y[0], test_size=0.25, shuffle=True, random_state=STATE)
X1_train, X1_valid, y1_train, y1_valid = train_test_split(X[1], y[1], test_size=0.25, shuffle=True, random_state=STATE)
X2_train, X2_valid, y2_train, y2_valid = train_test_split(X[2], y[2], test_size=0.25, shuffle=True, random_state=STATE)

In [15]:
# Проверка (для одного региона)

display(X0_train.shape, X0_valid.shape)
print('-' * 50)
display(y0_train.shape, y0_valid.shape)

(74991, 3)

(24998, 3)

--------------------------------------------------


(74991,)

(24998,)

In [16]:
# Масштабируем признаки методом стандартизации

X0_train, X0_valid = scaler_features(X0_train, X0_valid)
X1_train, X1_valid = scaler_features(X1_train, X1_valid)
X2_train, X2_valid = scaler_features(X2_train, X2_valid)

In [17]:
#?LinearRegression

Построим модели линейной регрессии по каждому региону:

In [18]:
model_0, y0_pred, score_0 = lr_model(X0_train, X0_valid, y0_train, y0_valid)
model_1, y1_pred, score_1 = lr_model(X1_train, X1_valid, y1_train, y1_valid)
model_2, y2_pred, score_2 = lr_model(X2_train, X2_valid, y2_train, y2_valid)

Для проверки модели на адекватность в каждом случае сравним с базовой моделью. В качестве базовой модели рассмотрим `DummyRegressor`, со стратегией `constant` (исп. среднее значение):

In [19]:
dr_score_0 = dr_model(X0_train, X0_valid, y0_train, y0_valid, y0_pred.mean())
dr_score_1 = dr_model(X1_train, X1_valid, y1_train, y1_valid, y1_pred.mean())
dr_score_2 = dr_model(X2_train, X2_valid, y2_train, y2_valid, y0_pred.mean())

In [20]:
# Для вывода результатов

result_evaluation_dict = {
        'RMSE for LinearReg': [score_0.round(3), score_1.round(3), score_2.round(3)],
        'Average stock': [y0_pred.mean().round(3), y1_pred.mean().round(3), y2_pred.mean().round(3)], 
        'RMSE for DummyReg': [dr_score_0.round(3), dr_score_1.round(3), dr_score_2.round(3)]
}

result_evaluation = pd.DataFrame(result_evaluation_dict, index=['geo_0', 'geo_1', 'geo_2'])
display(result_evaluation)

,RMSE for LinearReg,Average stock,RMSE for DummyReg
geo_0,37.729,92.610,44.325
geo_1,0.892,74.620,42.867
geo_2,40.326,95.052,45.084


#### Вывод:

Значение метрики модели линейной регрессии для региона `geo_1` существенно меньше по сравнению с остальными регионами, около 0.9 тыс.барр., что выглядит правдоподобно, учитывая очень сильную корреляцию по данному региону целевого признака с признаком `f2`.
Также видно, что RMSE моделей линейной регрессии ниже, чем у констаных моделей, что свидетельствует об их адекватности.

## Подготовка к расчёту прибыли

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе:

In [21]:
min_product_volume = BUDGET / NUM_WELLS / UNIT_INCOME
print('Достаточный объём сырья для безубыточной разработки новой скважины \
составляет {} тыс. баррелей'.format(round(min_product_volume, 3)))

Достаточный объём сырья для безубыточной разработки новой скважины составляет 111.111 тыс. баррелей


Получили, что по всем регионам средний запас предсказанного сырья ниже необходимого уровня для безубыточной разработки. Можно сделать вывод, что разработка любого из рассматриваемых регионов случайными 200 скважинами принесёт убытки.

Далее напишем функцию расчёта прибыли по выбранным скважинам и предсказаниям модели. На вход она принимает датафрейм (из двух столбцов, значений ц.п. на валидационной выборке и предсказаний) и кол-во скважин для разработки, возвращает значение прибыли, подсчитанное для скважин с максимальными значениями предсказаний:

In [22]:
# Функция расчёта прибыли

def income(df, count):
    
    top_pred = df.sort_values(by='predictions', ascending=False)
    selected = top_pred['target'][:count]
    
    return UNIT_INCOME * selected.sum() - BUDGET

## Расчёт прибыли и рисков 

Для определения распределения прибыли напишем функцию применения техники `Bootstrap` с 1000 выборками. На основе полученных данных рассчитаем среднюю прибыль, 95%-й доверительный интервал и риск убытков:

In [23]:
# Функция применения техники Bootstrap

def bootstrap_income(target, predictions):
    
    state = np.random.RandomState(STATE)
    
    target = target.reset_index(drop=True)
    predictions = pd.Series(predictions)
    
    df = pd.concat([target, predictions], axis=1)
    df.columns = ['target', 'predictions']
    
    values_income = []
    
    for _ in range(N_SAMP):
        df_sub = df.sample(n=N, replace=True, random_state=state)
        values_income.append(income(df_sub, NUM_WELLS))
        
    values_income = pd.Series(values_income)
    
    mean_income = values_income.mean()
    confidence_interval = [round(values_income.quantile(0.025) / 1000000, 1), 
                           round(values_income.quantile(0.975) / 1000000, 1)]
    risk_loss = values_income[values_income < 0].count() / values_income.shape[0] * 100
    
    return mean_income, confidence_interval, risk_loss

In [24]:
mean_income_0, confidence_interval_0, risk_loss_0 = bootstrap_income(y0_valid, y0_pred)
mean_income_1, confidence_interval_1, risk_loss_1 = bootstrap_income(y1_valid, y1_pred)
mean_income_2, confidence_interval_2, risk_loss_2 = bootstrap_income(y2_valid, y2_pred)

In [25]:
# Для вывода результатов

income_evaluation_dict = {
        'Average of income, mln. rub.': [round(mean_income_0/1000000, 1), round(mean_income_1/1000000, 1), 
                                         round(mean_income_2/1000000, 1)],
        'Interval': [confidence_interval_0, confidence_interval_1, confidence_interval_2], 
        'Risk of loss, %': [risk_loss_0, risk_loss_1, risk_loss_2]
}

income_evaluation = pd.DataFrame(income_evaluation_dict, index=['geo_0', 'geo_1', 'geo_2'])
display(income_evaluation)

,"Average of income, mln. rub.",Interval,"Risk of loss, %"
geo_0,418.0,"[-141.5, 932.9]",7.0
geo_1,678.6,"[277.8, 1094.1]",0.1
geo_2,344.5,"[-165.0, 873.3]",10.3


#### Вывод:

Получены данные по рискам убытков в трёх регионах, из которых видно, что на уровне меньше, чем 2.5%, находится только регион `geo_1`. Для этого же региона получена максимальная средняя прибыль. Данный регион может быть рекомендован для разработки месторождения.